In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
main_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/resume skills - assessment/CsvFiles/main_df_tokens.csv')

In [4]:
main_df.head()

,id,number_of_pages,text,category,pre_text,tokens,job_role
0,10554236,5,\nACCOUNTANT\nSummary\nFinancial Accountant sp...,ACCOUNTANT,accountant summary financial accountant specia...,"accountant,summary,financial,accountant,specia...",ACCOUNTANT
1,10674770,2,\nSTAFF ACCOUNTANT\nSummary\nHighly analytical...,ACCOUNTANT,staff accountant summary highly analytical det...,"staff,accountant,summary,highly,analytical,det...",STAFF ACCOUNTANT
2,11163645,2,\nACCOUNTANT\nProfessional Summary\nTo obtain ...,ACCOUNTANT,accountant professional summary obtain positio...,"accountant,professional,summary,obtain,positio...",ACCOUNTANT
3,11759079,2,\nSENIOR ACCOUNTANT\nExperience\nCompany Name\...,ACCOUNTANT,senior accountant experience company june curr...,"senior,accountant,experience,company,june,curr...",SENIOR ACCOUNTANT
4,12065211,2,\nSENIOR ACCOUNTANT\nProfessional Summary\nSen...,ACCOUNTANT,senior accountant professional summary senior ...,"senior,accountant,professional,summary,senior,...",SENIOR ACCOUNTANT


loading the total skills json

In [6]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [7]:
import spacy
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "/content/drive/MyDrive/Colab Notebooks/resume skills - assessment/JsonFiles/total_skills2.jsonl"

In [8]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [9]:
#save spacy model
nlp.to_disk('/content/drive/MyDrive/Colab Notebooks/resume skills - assessment/models/skills_extractor')

In [10]:
def get_skills_tokens(tokens):
    tokens=tokens.split(',')
    text=' '.join(tokens)
    doc = nlp(text)
    skill_set=set()
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            skill_set.add(ent.text)
    return ','.join(list(skill_set))

In [11]:
skills_final=main_df.loc[:][['id','category','tokens']]
skills_final.head()

,id,category,tokens
0,10554236,ACCOUNTANT,"accountant,summary,financial,accountant,specia..."
1,10674770,ACCOUNTANT,"staff,accountant,summary,highly,analytical,det..."
2,11163645,ACCOUNTANT,"accountant,professional,summary,obtain,positio..."
3,11759079,ACCOUNTANT,"senior,accountant,experience,company,june,curr..."
4,12065211,ACCOUNTANT,"senior,accountant,professional,summary,senior,..."


getting skills from tokens with final json
* it may take time

In [ ]:
skills_final['skills']=skills_final['tokens'].apply(get_skills_tokens)

In [ ]:
skills_final['skills_count_final']=skills_final['skills'].apply(lambda x:len(x.split(',')))

save the csv file

In [ ]:
skills_final.to_csv('/content/drive/MyDrive/Colab Notebooks/resume skills - assessment/CsvFiles/skills_final.csv',index=False)

In [18]:
skills_final.head()

,id,category,tokens,skills,skills_count_final
0,10554236,ACCOUNTANT,"accountant,summary,financial,accountant,specia...","document processing,deliver,data,safety,manage...",41
1,10674770,ACCOUNTANT,"staff,accountant,summary,highly,analytical,det...","advertising,word,design,data,financial,materia...",17
2,11163645,ACCOUNTANT,"accountant,professional,summary,obtain,positio...","business,word,payments,data,communications,wor...",13
3,11759079,ACCOUNTANT,"senior,accountant,experience,company,june,curr...","business,active,supervise,data,financial,accou...",11
4,12065211,ACCOUNTANT,"senior,accountant,professional,summary,senior,...","communication,schedule,financial,mode,material...",16


*  comparision between skills we got with different json files

In [12]:
skill_with_json1=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/resume skills - assessment/CsvFiles/skills_with_json1.csv')

In [20]:
compare=pd.concat([skill_with_json1[['id','category','skills_count_json1']],skills_final[['skills_count_final']]],axis=1)

In [21]:
compare.head()

,id,category,skills_count_json1,skills_count_final
0,10554236,ACCOUNTANT,20,41
1,10674770,ACCOUNTANT,8,17
2,11163645,ACCOUNTANT,6,13
3,11759079,ACCOUNTANT,5,11
4,12065211,ACCOUNTANT,7,16


In [26]:
def count_range(x):
    if x==0:
        return 'zero'
    elif x>0 and x<=10:
        return 'very less 1-10'
    elif x>10 and x<=15:
        return 'less 11-15'
    elif x>15 and x<=25:
        return 'ok 16-25'
    elif x>25 and x<=35:
        return 'enough 26-35'
    else:
        return 'more >35'

In [28]:
compare['range_json1']=compare['skills_count_json1'].apply(count_range)
compare['range_json2']=compare['skills_count_final'].apply(count_range)


In [29]:
compare.head()

,id,category,skills_count_json1,skills_count_final,range_json1,range_json2
0,10554236,ACCOUNTANT,20,41,ok 16-25,more >35
1,10674770,ACCOUNTANT,8,17,very less 1-10,ok 16-25
2,11163645,ACCOUNTANT,6,13,very less 1-10,less 11-15
3,11759079,ACCOUNTANT,5,11,very less 1-10,less 11-15
4,12065211,ACCOUNTANT,7,16,very less 1-10,ok 16-25


In [30]:
compare['range_json1'].value_counts()

very less 1-10    1866
less 11-15         377
ok 16-25           196
enough 26-35        32
more >35            12
Name: range_json1, dtype: int64

In [31]:
compare['range_json2'].value_counts()


ok 16-25          1080
less 11-15         577
very less 1-10     413
enough 26-35       317
more >35            96
Name: range_json2, dtype: int64

In [33]:
compare.head()

,id,category,skills_count_json1,skills_count_final,range_json1,range_json2
0,10554236,ACCOUNTANT,20,41,ok 16-25,more >35
1,10674770,ACCOUNTANT,8,17,very less 1-10,ok 16-25
2,11163645,ACCOUNTANT,6,13,very less 1-10,less 11-15
3,11759079,ACCOUNTANT,5,11,very less 1-10,less 11-15
4,12065211,ACCOUNTANT,7,16,very less 1-10,ok 16-25


In [ ]:
compare.to_csv('/content/drive/MyDrive/Colab Notebooks/resume skills - assessment/CsvFiles/compare.csv',index=False)